In [1]:
pip install pandas torch transformers deepchem


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 12.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel
import torch

In [3]:

# Load ChemBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
model = RobertaModel.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/9.43k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/3.21k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/179M [00:00<?, ?B/s]

In [4]:


# Load dataset with SMILES column
df = pd.read_csv('smiles_dataset.csv')
smiles_list = df['smiles'].tolist()

# Load ChemBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
model = RobertaModel.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')

# Create an empty list to store embeddings
smiles_embeddings = []

# Loop over each SMILES in the dataset
for smiles in smiles_list:
    # Tokenize the SMILES string
    inputs = tokenizer(smiles, return_tensors='pt')

    # Generate embeddings using ChemBERTa model
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the last hidden state as embeddings
    embeddings = outputs.last_hidden_state

    # Convert the embeddings to a numpy array and flatten
    embedding_numpy = embeddings.detach().numpy().flatten()

    # Append to the embeddings list
    smiles_embeddings.append(embedding_numpy)

# Add the embeddings to the dataframe
df['embeddings'] = smiles_embeddings


df.to_csv('smiles_with_embeddings.csv', index=False)

print("Embeddings generated and saved successfully!")



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embeddings generated and saved successfully!


In [5]:
# Get the output embeddings
outputs = model(**inputs)

# The hidden state at the last layer
embeddings = outputs.last_hidden_state


In [6]:
embeddings.shape


torch.Size([1, 5, 768])